# 空间连接

本节目标:

- 基于 `countries` 和 `cities` dataframes, 按照所在国家给每个城市分类.
- 为了解决这个问题, 我们会使用一个概念'空间连接'操作: 基于空间关系合并空间数据集.

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas

pd.options.display.max_rows = 10

In [ ]:
countries = geopandas.read_file("zip://./data/ne_110m_admin_0_countries.zip")
cities = geopandas.read_file("zip://./data/ne_110m_populated_places.zip")
rivers = geopandas.read_file("zip://./data/ne_50m_rivers_lake_centerlines.zip")

## 概要 - 连接 dataframes

Pandas 提供多重连接或者合并 dataframes 的功能

查看简介： https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/

完整文档： https://pandas.pydata.org/pandas-docs/stable/merging.html

演示下如何使用 pandas 连接两个 dataframes, 我们分别从 `cities` 和 `countries` datasets 中取子集: 

In [ ]:
cities2 = cities[cities['name'].isin(['Bern', 'Brussels', 'London', 'Paris'])].copy()
cities2['iso_a3'] = ['CHE', 'BEL', 'GBR', 'FRA']

In [ ]:
cities2

In [ ]:
countries2 = countries[['iso_a3', 'name', 'continent']]
countries2.head()

我们在 `cities` 数据集上增加了增加了 `iso_a3` 列, 用来标识该城市所在的国家. 这个国家代码也出现在 `countries` 数据集, 使我们可以基于这个共同的列来合并两个 dataframes.

合并 `cities` 和 `countries` 会基于共同的键把国家的一些额外信息 (全称, 大陆) 传递给 `cities` dataframe:

In [ ]:
cities2.merge(countries2, on='iso_a3')

**但是**, 在这个例子中, 我们手工添加了这一列, 它并不在我们的原始数据中. 然而, 我们会知道如何基于空间坐标来连接两个 dataframes.

## 概要 - 对象间的空间关系

在上一个 notebook 中 [02-spatial-relationships.ipynb](./02-spatial-relationships-operations.ipynb),我们看到了空间关系相关的概念: 包含, 被包含, 交叉, ...

在这个例子中, 我们知道每个城市都*被包含*在一个国家里, 另一方面每个国家可以*包含*多个城市.

我们可以用上节中见过的方法来测试这个关系:

In [ ]:
france = countries.loc[countries['name'] == 'France', 'geometry'].squeeze()

In [ ]:
cities.within(france)

上面的语句返回一个布尔值组成的序列, 用来表面 `cities` dataframe 中每个点是否被包含于法国范围内.  
因为这是个布尔序列, 所以我们可以用它来过滤原始数据中真正位于发过的城市:

In [ ]:
cities[cities.within(france)]

我们现在可以对每个国家重复上面的分析, 然后在 `cities` dataframe 增加一列来标识国家.然而,手工操作过于乏味,而且空间连接运算给我们提供了相关功能.

*(提醒: 因为国家数据集比较粗糙，上面的结果有错误)*

## 空间连接运算

<div class="alert alert-info" style="font-size:120%">
<b>空间连接</b> = *基于空间关系把属性从一层转移到另一层* <br><br>


这个运算的与众不同之处:

<ul>
  <li>想要在 GeoDataFrame 中添加信息</li>
  <li>GeoDataFrame 里包含我们想要添加的信息 </li>
  <li>两个我们需要使用的数据集需要满足空间关系 ('交叉', '包含', '被包含')</li>
  <li>连接类型: left join 或者 inner join</li>
</ul>

</div>

在这个例子中, 我们想基于空间关系把 `cities` dataframe 和 `countries` dataframe 连接.

我们使用 [`geopandas.sjoin`](http://geopandas.readthedocs.io/en/latest/reference/geopandas.sjoin.html) 函数:

In [ ]:
joined = geopandas.sjoin(cities, countries, op='within', how='left')

In [ ]:
joined

In [ ]:
joined['continent'].value_counts()

## 覆盖运算

在上面的空间连接运算中,我们没有改变几何图形本身.我们没有连接几何图形,而是基于几何图形的空间联系连接了其他属性.这表示这些图形至少有一部分使重叠的.

如果你想通过连接空间数据集来创造新的几何图形 (比如对几何图形取交集), 你需要 **overlay** 操作.

In [ ]:
africa = countries[countries['continent'] == 'Africa']

In [ ]:
africa.plot()

In [ ]:
cities['geometry'] = cities.buffer(2)

In [ ]:
geopandas.overlay(africa, cities, how='difference').plot()

<div class="alert alert-info" style="font-size:120%">
<b>牢记</b> <br>

<ul>
  <li>**空间连接**: 基于空间关系转移属性</li>
  <li>**空间覆盖**: 基于两个 dataframes 之间的空间运算创建新的jie (and combining attributes of both dataframes)</li>
</ul>

</div>